In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import warnings
import re

In [2]:
pip install pytorch-tabnet;

In [3]:
pip install optuna;

In [4]:
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold,cross_val_score
from sklearn.linear_model import LassoCV,LassoLarsIC,Lasso,LinearRegression

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
def lasso_moran_select(data,s):

    X_coords = data.iloc[:,:-3]

    y = data['y']

    X1 = data.X1.values
    X2 = data.X2.values

    moran = data.iloc[:,2:-3].values

    moran_X1_names = ["moran_X1_" + str(i) for i in range(moran.shape[1])]
    X_coords.loc[:,moran_X1_names] = moran*X1.reshape(-1,1)

    moran_X2_names = ["moran_X2_" + str(i) for i in range(moran.shape[1])]
    X_coords.loc[:,moran_X2_names] = moran*X2.reshape(-1,1)

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    if s == 'mse':
        model = LassoCV(cv=kf,  alphas=np.logspace(-4, -2, 20))
        model.fit(X_coords, y)
        r2_scores = cross_val_score(model, X_coords, y, cv=kf, scoring='r2')

        print("MSE LASSO CV R2 score: ", np.mean(r2_scores))

    if s == 'bic':
        model = LassoLarsIC(criterion='bic')
        model.fit(X_coords, y)

        r2_scores = cross_val_score(model, X_coords, y, cv=kf, scoring='r2')

        print("BIC LASSO CV R2 score: ", np.mean(r2_scores))


    selected_index = np.where(model.coef_ != 0)[0]
    selected_feature = list(X_coords.columns[selected_index])

    moran_X1s = [s for s in selected_feature if s.startswith('moran_X1_')]
    moran_X2s = [s for s in selected_feature if s.startswith('moran_X2_')]
    moran_s = [s for s in selected_feature if re.match(r"^moran_\d+$", s)]

    numbers_s = set(int(x.split('_')[1]) for x in moran_s)
    numbers_x1 = set(int(x.split('_')[2]) for x in moran_X1s)
    numbers_x2 = set(int(x.split('_')[2]) for x in moran_X2s)

    # Find intersection of numbers in both lists
    common_numbers = numbers_s.union(numbers_x1.union(numbers_x2))

    moran_selected = ['moran_' + str(num) for num in sorted(common_numbers)]

    return moran_selected

In [7]:
import sys
sys.path.append('/content/drive/MyDrive/MEM Colab Runs')

In [8]:
from tabnet_model import TabNet_fit

In [9]:
weights = ['queen','exp'] #2
models = ['tabnet'] #1
selection = ['xy','all','mse','bic'] #4

In [10]:
warnings.filterwarnings('ignore')

for w in weights:
    data = pd.read_csv('/content/drive/MyDrive/MEM Colab Runs/data/us_moran_' + w + '.csv',index_col=0)

    for s in selection:

        if s == 'all':
            selected = list(data.columns[2:-3])
        if s == 'xy':
            selected = list(data.columns[-3: -1])
        if s =='mse':
            selected = lasso_moran_select(data,s)
        if s == 'bic':
            selected = lasso_moran_select(data,s)
        if s == 'aic':
            selected = lasso_moran_select(data,s)

        X_coords = data[['X1','X2']  + selected]

        moran = X_coords[selected]
        moran = (np.array(moran) - np.array(moran).mean(axis=0))/np.array(moran).std(axis=0)
        X_coords[selected] = moran

        y = data['y']

        for m in models:

            model,best_loss = TabNet_fit(X_coords.values, y.values.reshape(-1,1), timeout=60*30)

            #model.fit(X_coords, y, estimator_list = [m], **settings)

            name = "/content/drive/MyDrive/MEM Colab Runs/models/us_" + m + "_moran_" + s + "_lasso_" + w + ".model"

            pickle.dump(model, open(name, 'wb'))

            print(name + ' CV-R2 = {0:.3g}'.format(best_loss))

[I 2024-05-10 11:21:30,635] A new study created in memory with name: no-name-81229f5c-80a6-4bf5-9346-d2aea5b45b21



Early stopping occurred at epoch 365 with best_epoch = 293 and best_val_0_rmse = 0.75267

Early stopping occurred at epoch 258 with best_epoch = 186 and best_val_0_rmse = 0.92831
Stop training because you reached max_epochs = 369 with best_epoch = 363 and best_val_0_rmse = 0.80237

Early stopping occurred at epoch 335 with best_epoch = 263 and best_val_0_rmse = 0.76754


[I 2024-05-10 11:25:27,931] Trial 0 finished with value: -0.938029325254638 and parameters: {'mask_type': 'sparsemax', 'n_da': 64, 'n_steps': 3, 'gamma': 1.1, 'n_shared': 1, 'lambda_sparse': 1.951722464144948e-08, 'patienceScheduler': 14, 'patience': 72, 'epochs': 369}. Best is trial 0 with value: -0.938029325254638.



Early stopping occurred at epoch 325 with best_epoch = 253 and best_val_0_rmse = 0.96348
Stop training because you reached max_epochs = 181 with best_epoch = 174 and best_val_0_rmse = 0.88067
Stop training because you reached max_epochs = 181 with best_epoch = 179 and best_val_0_rmse = 0.95218
Stop training because you reached max_epochs = 181 with best_epoch = 180 and best_val_0_rmse = 1.04004
Stop training because you reached max_epochs = 181 with best_epoch = 176 and best_val_0_rmse = 1.07279


[I 2024-05-10 11:27:08,364] Trial 1 finished with value: -0.9156999934520623 and parameters: {'mask_type': 'sparsemax', 'n_da': 64, 'n_steps': 2, 'gamma': 1.1, 'n_shared': 1, 'lambda_sparse': 3.3205591037519537e-07, 'patienceScheduler': 9, 'patience': 60, 'epochs': 181}. Best is trial 0 with value: -0.938029325254638.


...


Early stopping occurred at epoch 173 with best_epoch = 138 and best_val_0_rmse = 0.85884
Stop training because you reached max_epochs = 102 with best_epoch = 69 and best_val_0_rmse = 1.52728
Stop training because you reached max_epochs = 102 with best_epoch = 100 and best_val_0_rmse = 1.21663
Stop training because you reached max_epochs = 102 with best_epoch = 101 and best_val_0_rmse = 1.26211
Stop training because you reached max_epochs = 102 with best_epoch = 100 and best_val_0_rmse = 1.54504


[I 2024-05-10 15:37:21,864] Trial 7 finished with value: -0.8334061421868354 and parameters: {'mask_type': 'entmax', 'n_da': 56, 'n_steps': 5, 'gamma': 1.4, 'n_shared': 4, 'lambda_sparse': 7.183758655813986e-05, 'patienceScheduler': 3, 'patience': 55, 'epochs': 102}. Best is trial 6 with value: -0.9466092036236742.


Stop training because you reached max_epochs = 102 with best_epoch = 97 and best_val_0_rmse = 1.37419

Early stopping occurred at epoch 194 with best_epoch = 102 and best_val_0_rmse = 0.75775

Early stopping occurred at epoch 193 with best_epoch = 101 and best_val_0_rmse = 0.88277

Early stopping occurred at epoch 212 with best_epoch = 120 and best_val_0_rmse = 0.79719

Early stopping occurred at epoch 212 with best_epoch = 120 and best_val_0_rmse = 0.76596


[I 2024-05-10 15:39:04,824] Trial 8 finished with value: -0.9449054421510116 and parameters: {'mask_type': 'entmax', 'n_da': 56, 'n_steps': 1, 'gamma': 1.2, 'n_shared': 2, 'lambda_sparse': 4.446628955475447e-05, 'patienceScheduler': 11, 'patience': 92, 'epochs': 262}. Best is trial 6 with value: -0.9466092036236742.



Early stopping occurred at epoch 245 with best_epoch = 153 and best_val_0_rmse = 0.78503
/content/drive/MyDrive/MEM Colab Runs/models/us_tabnet_moran_bic_lasso_exp.model CV-R2 = 0.947


### LASSO Results

In [12]:
for w in weights:
    data = pd.read_csv('/content/drive/MyDrive/MEM Colab Runs/data/us_moran_' + w + '.csv',index_col=0)

    for s in ['mse','bic','aic']:

        if s =='mse':
            selected = lasso_moran_select(data,s)
            print(len(selected))
        if s == 'bic':
            selected = lasso_moran_select(data,s)
            print(len(selected))

MSE LASSO CV R2 score:  0.79067547896043
177
BIC LASSO CV R2 score:  0.7601509265296883
115
MSE LASSO CV R2 score:  0.8404520991224775
88
BIC LASSO CV R2 score:  0.8331425500911098
44


### Linear Model

In [15]:
X_coords = data.iloc[:,:2]

model = LinearRegression()

# Set up cross-validation strategy
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and calculate R2 scores
r2_scores = cross_val_score(model, X_coords, y, cv=kf, scoring='r2')

In [16]:
r2_scores.mean()

0.47091006560697685